In [ ]:
#Importing packgaes
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from google.colab import drive
from sklearn.feature_extraction.text import TfidfTransformer
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# read json into a dataframe
df_idf=pd.read_json("/content/gdrive/MyDrive/stackoverflow-data-idf.json",lines=True)

In [ ]:
#Function to pre-process the documents, in order 
#to conert them into lower case, removal of tags special character and digits.
def pre_process(text):
    
    # lowercase
    text=text.lower()
    
    #remove tags
    text=re.sub("</?.*?>"," <> ",text)
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    return text


In [ ]:
#Only title and body is needed, so new dataframe is created and pre-processed
df_idf['text'] = df_idf['title'] + df_idf['body']
df_idf['text'] = df_idf['text'].apply(lambda x:pre_process(x))


In [ ]:
df_idf['text'].head()

0    serializing a private struct can it be done i ...
1    how do i prevent floated right content from ov...
2    gradle command line i m trying to run a shell ...
3    loop variable as parameter in asynchronous fun...
4    canot get the href value hi i need to valid th...
Name: text, dtype: object

In [ ]:
def get_stop_words(stop_file_path):
    """load stop words """
    
    with open(stop_file_path, 'r', encoding="utf-8") as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        return frozenset(stop_set)

#load a set of stop words
stopwords=get_stop_words("/content/gdrive/MyDrive/stopwords.txt")

#get the text column 
docs=df_idf['text'].tolist()

#create a vocabulary of words, 
#ignore words that appear in 85% of documents, 
#eliminate stop words
cv=CountVectorizer(max_df=0.85,stop_words=stopwords)
word_count_vector=cv.fit_transform(docs)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['come', 'vis', 'viser', 'visest'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [ ]:
#Making the count vector with vocabulary size 10000
cv=CountVectorizer(max_df=0.85,stop_words=stopwords,max_features=10000)
word_count_vector=cv.fit_transform(docs)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['come', 'vis', 'viser', 'visest'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [ ]:
#Creation of tf-idf vector using created count vector
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [ ]:
# read test docs into a dataframe and concatenate title and body
df_test=pd.read_json("/content/gdrive/MyDrive/stackoverflow-test.json",lines=True)
df_test['text'] = df_test['title'] + df_test['body']
df_test['text'] =df_test['text'].apply(lambda x:pre_process(x))

# get test docs into a list
docs_test=df_test['text'].tolist()
docs_title=df_test['title'].tolist()
docs_body=df_test['body'].tolist()

In [ ]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [ ]:
def getkeywords(idx):

    #generate tf-idf for the given document
    tf_idf_vector=tfidf_transformer.transform(cv.transform([docs_test[idx]]))

    #sort the tf-idf vectors by descending order of scores
    sorted_items=sort_coo(tf_idf_vector.tocoo())

    #extract only the top n; n here is 10
    keywords=extract_topn_from_vector(cv.get_feature_names(),sorted_items,10)
    
    return keywords

def print_results(idx,keywords):
    # now print the results
    print("\n=====Title=====")
    print(docs_title[idx])
    print("\n=====Body=====")
    print(docs_body[idx])
    print("\n===Keywords===")
    for k in keywords:
        print(k,keywords[k])


Extracting top 10 keywords for each sample

In [ ]:
for i in range(len(df_test)):
  keywords= getkeywords(i)
  print_results(i,keywords)
  print("\n\n")

Streaming output truncated to the last 5000 lines.
=====Title=====
React Native SQLite wrapper for trigonometric function support such cos, acos ,sin etc in queries

=====Body=====
<p>I am using react-native-sqlite-storage wrapper for my data storage in react native ios app.i have a requirement of fetching nearby location data based on current location which can be achieved using distance finding formula haversine formula for which trigonometric functions are used.But SQLite does not support trigonometric function(acos,cos,sin) etc and getting error acos function not found while using in a query. 
Any help would be appreciated!
Thanks!</p>

===Keywords===
sqlite 0.378
react 0.367
native 0.309
cos 0.298
sin 0.289
formula 0.266
wrapper 0.238
storage 0.238
support 0.186
location 0.178




=====Title=====
Linking to ffmpeg libraries fails while installing OpenCV

=====Body=====
<p>FFMpeg 2.7.2 and all necessary <code>libav*</code> libraries are installed on my Ubuntu 14.04 machine. However